In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [3]:
from dotenv import load_dotenv
import nest_asyncio
import os

os.environ["SWE_AGENT_CONFIG_DIR"] = "."
os.environ["SWE_AGENT_TOOLS_DIR"] = "tools"
os.environ["SWE_AGENT_TRAJECTORY_DIR"] = "trajectories"

os.makedirs("replays", exist_ok=True)
os.makedirs("trajectories", exist_ok=True)

load_dotenv()
nest_asyncio.apply()

In [4]:
import art
from art.local import LocalBackend
from rollout import ModelConfig

backend = LocalBackend()
model = art.TrainableModel(
    name="001",
    project="sweagent",
    config=ModelConfig(
        max_input_tokens=32_768,
        system_prompt_suffix="\n/no_think",
        xml_function_calling=True,
    ),
    base_model="Qwen/Qwen3-32B",
)
await model.register(backend)

INFO 06-03 17:52:44 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-03 17:52:44 [__init__.py:239] Automatically detected platform cuda.


/home/sky/sky_workdir/src/art/__init__.py:11: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth  # type: ignore


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-03 17:52:51 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-03 17:52:51 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.1: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA H200. Num GPUs = 1. Max memory: 139.719 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen3-32b-bnb-4bit with actual GPU utilization = 78.66%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 139.72 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill toke

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.51it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.43it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.34it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.35it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.37it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.47it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.38it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.33it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.31it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.33i

INFO 06-03 17:53:08 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 06-03 17:53:08 [model_runner.py:1140] Model loading took 18.1555 GiB and 6.536463 seconds
INFO 06-03 17:53:14 [worker.py:287] Memory profiling takes 5.72 seconds
INFO 06-03 17:53:14 [worker.py:287] the current vLLM instance can use total_gpu_memory (139.72GiB) x gpu_memory_utilization (0.79) = 109.90GiB
INFO 06-03 17:53:14 [worker.py:287] model weights take 18.16GiB; non_torch_memory takes 0.15GiB; PyTorch activation peak memory takes 6.53GiB; the rest of the memory reserved for KV Cache is 85.07GiB.
INFO 06-03 17:53:15 [executor_base.py:112] # cuda blocks: 21777, # CPU blocks: 1536
INFO 06-03 17:53:15 [executor_base.py:117] Maximum concurrency for 32768 tokens per request: 10.63x
INFO 06-03 17:53:18 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the C

Capturing CUDA graph shapes: 100%|█████████████| 53/53 [00:49<00:00,  1.06it/s]


INFO 06-03 17:54:08 [model_runner.py:1592] Graph capturing finished in 50 secs, took 3.52 GiB
INFO 06-03 17:54:08 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 59.12 seconds
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']


Unsloth 2025.5.1 patched 64 layers with 64 QKV layers, 64 O layers and 64 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


In [5]:
import asyncio
import modal

sandboxes: list[modal.Sandbox] = []
async for sandbox in modal.Sandbox.list.aio(
    app_id=modal.App.lookup("swe-rex", create_if_missing=True).app_id
):
    sandboxes.append(sandbox)
_ = await asyncio.gather(*[sandbox.terminate.aio() for sandbox in sandboxes])

In [ ]:
from instances import as_instances_iter, get_filtered_swe_smith_instances_df
from rollout import rollout

tasks = (
    get_filtered_swe_smith_instances_df()
    .sample(fraction=1.0, shuffle=True, seed=42)
    .pipe(as_instances_iter)
)

async for batch in art.trajectory_group_batches(
    (art.TrajectoryGroup(rollout(model, task) for _ in range(4)) for task in tasks),
    batch_size=4,
    # max_batch_exceptions=128,
    max_concurrent_batches=4,
    skip_batches=await model.get_step(),
):
    await model.train(batch)

batches:   0%|          | 0/16 [00:00<?, ?it/s]

eval_result: {'num_failed_f2p': 0, 'num_passed_f2p': 0, 'num_failed_p2p': 0, 'num_passed_p2p': 0}, instance_id: facebookresearch__hydra.0f03eb60.lm_rewrite__djqcu4h3
Container process terminated.
stdout:
INFO:     172.20.0.1:36618 - "GET /is_alive HTTP/1.1" 200 OK
INFO:     172.20.0.1:36622 - "POST /create_session HTTP/1.1" 200 OK
INFO:     172.20.0.1:36634 - "POST /run_in_session HTTP/1.1" 200 OK
INFO:     172.20.0.1:36646 - "POST /run_in_session HTTP/1.1" 200 OK
INFO:     172.20.0.1:36662 - "POST /run_in_session HTTP/1.1" 200 OK
INFO:     172.20.0.1:36668 - "POST /run_in_session HTTP/1.1" 200 OK
INFO:     172.20.0.1:36680 - "POST /run_in_session HTTP/1.1" 200 OK
INFO:     172.20.0.1:36680 - "POST /run_in_session HTTP/1.1" 200 OK
INFO:     172.20.0.1:36680 - "POST /run_in_session HTTP/1.1" 200 OK
INFO:     172.20.0.1:36680 - "POST /run_in_session HTTP/1.1" 200 OK
INFO:     172.20.0.1:36680 - "POST /run_in_session HTTP/1.1" 200 OK
INFO:     172.20.0.1:36680 - "POST /run_in_session HTTP

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bradhilton to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Wandb run initialized! You can view it at https://wandb.ai/bradhilton/sweagent/runs/001


tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

There are no assistant logprobs to train on. Did you forget to include at least one Choice in Trajectory.messages_and_choices?
Skipping tuning as there is no suitable data. This can happen when all the trajectories in the same group have the same reward and thus no advantage to train on.


batches:   0%|          | 0/16 [00:00<?, ?it/s]

HTTPSConnectionPool(host='d41ivkzszkf4qg.r38.modal.host', port=443): Max retries exceeded with url: /run_in_session (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1007)')))
HTTPSConnectionPool(host='gy06fbrsmu19nx.r37.modal.host', port=443): Max retries exceeded with url: /run_in_session (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1007)')))
HTTPSConnectionPool(host='9nupbp45qvfagr.r30.modal.host', port=443): Max retries exceeded with url: /run_in_session (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1007)')))
HTTPSConnectionPool(host='1t37ip7a0a4rmo.r33.modal.host', port=443): Max retries exceeded with url: /run_in_session (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1007)')))
